# Mandatory Assignment 3


## US Jobs and Import Competition from China 
Af: Mathias Nikolaisen(dnt357), Oliver Mattthews(jvc975) & Clara Maj Folkmann(djz279)

### Problem 1

In [ ]:
import pandas as pd
import statsmodels.api as sm
import numpy as np

: 

### Problem 2

                            OLS Regression Results                            
Dep. Variable:                aarslon   R-squared:                       0.073
Model:                            OLS   Adj. R-squared:                  0.072
Method:                 Least Squares   F-statistic:                     125.4
Date:                Sun, 06 Oct 2024   Prob (F-statistic):           3.61e-78
Time:                        14:20:29   Log-Likelihood:                -4102.0
No. Observations:                4800   AIC:                             8212.
Df Residuals:                    4796   BIC:                             8238.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       -457.8689     38.818    -11.795      0.0

### Problem 3

                            OLS Regression Results                            
Dep. Variable:             ln_aarslon   R-squared:                       0.125
Model:                            OLS   Adj. R-squared:                  0.124
Method:                 Least Squares   F-statistic:                     114.6
Date:                Sun, 06 Oct 2024   Prob (F-statistic):          1.48e-135
Time:                        14:52:20   Log-Likelihood:                -3961.5
No. Observations:                4800   AIC:                             7937.
Df Residuals:                    4793   BIC:                             7982.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const             8.0633      0.128     62.821

## Problem 4

## Problem 5